# 第7回課題
2段階の train, valid, test にわける交差検証の自前の実装
（パッケージにしたり関数にしたりするのは自由です．解答はしていません）

https://scikit-learn.org/stable/modules/cross_validation.html　で様々なパターンについて述べられています

# 注意事項
train, valid, test で取ってくるサンプル（id）が一致しないということを必ず確認してください

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import random
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()

In [2]:
data = load_wine()

In [3]:
wine_df = pd.DataFrame(min_max_scaler.fit_transform(data["data"]), columns=data["feature_names"])
wine_df = pd.concat([wine_df, pd.DataFrame(data["target"], columns=["target"])], axis=1)
print(wine_df.shape)
wine_df.head()

(178, 14)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,0.842105,0.191700,0.572193,0.257732,0.619565,0.627586,0.573840,0.283019,0.593060,0.372014,0.455285,0.970696,0.561341,0
1,0.571053,0.205534,0.417112,0.030928,0.326087,0.575862,0.510549,0.245283,0.274448,0.264505,0.463415,0.780220,0.550642,0
2,0.560526,0.320158,0.700535,0.412371,0.336957,0.627586,0.611814,0.320755,0.757098,0.375427,0.447154,0.695971,0.646933,0
3,0.878947,0.239130,0.609626,0.319588,0.467391,0.989655,0.664557,0.207547,0.558360,0.556314,0.308943,0.798535,0.857347,0
4,0.581579,0.365613,0.807487,0.536082,0.521739,0.627586,0.495781,0.490566,0.444795,0.259386,0.455285,0.608059,0.325963,0


# モデル（今回は決定木を利用）

In [4]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state=0)

# seed の設定

In [23]:
seed = 0
random.seed(seed)
np.random.seed(seed)

# k-fold の k は変数で扱うこと

In [6]:
k = 5

# train (valid 含む), test の分割

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(wine_df[data["feature_names"]], wine_df["target"], test_size=0.33, random_state=1)

# 交差検証
train, valid を分割しつつ valid をテスト

In [56]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=k, shuffle=True)
y_train[66]
X_train.loc[66]
a=[0,1,2,3]
y_train[a]
X_train

0    0
1    0
2    0
3    0
Name: target, dtype: int32

In [19]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(clf,X_train,y_train,cv=k)
score

array([0.83333333, 0.95833333, 0.91666667, 0.875     , 0.82608696])

In [18]:
cross_val_score(clf,X_train,y_train,cv=kf)

array([0.75      , 0.875     , 0.95833333, 1.        , 0.86956522])

In [99]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [101]:
# k回訓練し，valid で評価
scores = cross_val_score(clf, X_train, y_train, cv=k)
scores

array([0.83333333, 0.95833333, 0.91666667, 0.875     , 0.82608696])

In [ ]:
scores = cross_val_score(clf, X_train, y_train, cv=k)
scores

In [ ]:
# train すべてで訓練し，test を評価
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
accuracy_score(pred, y_test)

In [113]:
def cross_validate(x,y, split_size=5):
    results = []
    kf = KFold(n_splits=split_size)
    
    for train_idx, val_idx in kf.split(x, y):
        print(34343)
        train_x = x[train_idx]
        print(4444)
        train_y = y[train_idx]
        val_x = x[val_idx]
        val_y = y[val_idx]

In [112]:
cross_validate(X_train, y_train)

34343


KeyError: "None of [Int64Index([ 24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,\n             37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,\n             50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,\n             63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,\n             76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,\n             89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,\n            102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,\n            115, 116, 117, 118],\n           dtype='int64')] are in the [columns]"

In [102]:
from sklearn.model_selection import KFold

# (snip)

def cross_validate(session, split_size=5):
  results = []
  kf = KFold(n_splits=split_size)
  for train_idx, val_idx in kf.split(train_x_all, train_y_all):
    train_x = train_x_all[train_idx]
    train_y = train_y_all[train_idx]
    val_x = train_x_all[val_idx]
    val_y = train_y_all[val_idx]
    run_train(session, train_x, train_y)
    results.append(session.run(accuracy, feed_dict={x: val_x, y: val_y}))
  return results

# (snip)

In [104]:
with tf.Session() as session:
  result = cross_validate(session)
  print("Cross-validation result: %s" % result)
  print("Test accuracy: %f" % session.run(accuracy, feed_dict={x: test_x, y: test_y})) #<- ここ

NameError: name 'tf' is not defined

In [80]:
for train_index, val_index in kf.split(X_train, y_train):
    model = build_model()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x=X_train[train_index], y=_y_train[train_index], batch_size=100, epochs=10, verbose=1)
    _history.append(model.evaluate(x=X_train[val_index], y=_y_train[val_index], batch_size=100))

NameError: name 'build_model' is not defined

In [95]:
def myloc(x_df, index):
    df=x_df.iloc[index[0]]
    for i in index[1:]:
        df=df.append(x_df.iloc[i])
    return df

In [98]:
clf.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [97]:
for train_index, val_index in kf.split(X_train, y_train):
    clf.fit(myloc(X_train, train_index), y_train[train_index])


KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'